<a href="https://colab.research.google.com/github/jiteshpabla/tweet-sentiment-extraction/blob/main/tweets_sentiment_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
DIR = "/content/drive/My Drive/kaggle/twitter_sentiment_extraction/"

In [ ]:
import pandas as pd

df = pd.read_csv(DIR+"tweet-sentiment-extraction/train.csv")
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [ ]:
df.dropna(inplace=True)
df

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative
...,...,...,...,...
27476,4eac33d1c0,wish we could come see u on Denver husband l...,d lost,negative
27477,4f4c4fc327,I`ve wondered about rake to. The client has ...,", don`t force",negative
27478,f67aae2310,Yay good for both of you. Enjoy the break - y...,Yay good for both of you.,positive
27479,ed167662a5,But it was worth it ****.,But it was worth it ****.,positive


In [ ]:
text = df['text'].values
selected_text = df['selected_text'].values
sentiment = df['sentiment'].values

In [ ]:
!pip install transformers

     |████████████████████████████████| 645kB 5.9MB/s 
     |████████████████████████████████| 1.1MB 16.4MB/s 
     |████████████████████████████████| 890kB 37.6MB/s 
     |████████████████████████████████| 3.8MB 25.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=3d08875bb15de35970ea1ea2a66e3147841fcf86e8d2ad490481a86fabdbfe9c
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


Different NLP algorithms require different types of tokenizations of the input word sequence. The BERT architecture requires the input words to be Wordpiece tokenized. 'BertTokenizer' does this for us.

In [ ]:
from transformers import BertTokenizer

MODEL_NAME = "bert-base-uncased" #DIR+"QA_model2/"#"bert-base-cased"

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME, do_lower_case=True)

In [ ]:
print(len(text))
print(len(selected_text))

print(tokenizer.tokenize(text[0], add_special_tokens=True))
print(tokenizer.tokenize(selected_text[0], add_special_tokens=True))

27480
27480
['i', '`', 'd', 'have', 'responded', ',', 'if', 'i', 'were', 'going']
['i', '`', 'd', 'have', 'responded', ',', 'if', 'i', 'were', 'going']


In [ ]:
# max_len = 0
# for sent in text:
#     input_id = tokenizer.encode(sent, add_special_tokens=True)
#     max_len = max(max_len, len(input_id))
# print(max_len)

# Output is 110

In [ ]:
import torch

input_ids = []
attention_masks = []
token_type_ids = []

for i in range(len(text)):


    '''question = "What signifies a "+sentiment[i]+" sentiment in this: "text[i]+"?"
    encoding = tokenizer.encode_plus(question, text[i])
    input_ids, token_type_ids = encoding["input_ids"], encoding["token_type_ids"]
    start_scores, end_scores = model(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))

    all_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    answer = ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1])'''

    question = "What signifies a "+sentiment[i]+" sentiment in the following sentence?

    encoded = tokenizer.encode_plus(
      question,
      text[i],
      add_special_tokens=True,
      max_length=150,
      pad_to_max_length=True,
      return_token_type_ids=True,
      return_attention_mask=True,
      return_tensors='pt'
    )
    
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    token_type_ids.append(encoded['token_type_ids'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)

print(input_ids[0])
print(attention_masks[0])
print(token_type_ids[0])
print(token_type_ids[0:2])

print(len(input_ids[0]))



tensor([ 101, 8699,  102, 1045, 1036, 1040, 2031, 5838, 1010, 2065, 1045, 2020,
        2183,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [ ]:
import numpy as np

input_ids = input_ids.numpy()
attention_masks = attention_masks.numpy()
token_type_ids = token_type_ids.numpy()


start_pos = []
end_pos = []
count = 0

reject_list = []

length_text = len(input_ids)
print(length_text)

i = 0
while i < length_text:
    
    text_ids = input_ids[i]
    print("text_ids", text_ids)
    selected_text_ids = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(selected_text[i], add_special_tokens=True))
    print("selected_text_ids", selected_text_ids)
    first = selected_text_ids[0]
    
    if len(selected_text_ids) == 1:
        second = -1
    else:
        second = selected_text_ids[1]
    
    pos = -1
    ctr = -1
    for j in range(len(text_ids) - 1):
        pos += 1
        if second == -1:
            if text_ids[j] == first:
                ctr *= -1
                start_pos.append(pos)
                break
        else:
            if text_ids[j] == first and text_ids[j+1] == second:
                ctr *= -1
                start_pos.append(pos)
                break
    if ctr == -1:
        count += 1
        
        ##
        selected_text = np.delete(selected_text, i)
        input_ids = np.delete(input_ids, i, axis=0)
        attention_masks = np.delete(attention_masks, i, axis=0)
        token_type_ids = np.delete(token_type_ids, i, axis=0)
        
        length_text -= 1
        i -= 1
    else:
        end_pos.append(pos + len(selected_text_ids) - 1)
    i +=1

print("count", count)

Streaming output truncated to the last 5000 lines.
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0]
selected_text_ids [3038, 9119, 2000, 13008, 1012, 1012, 1012, 1012]
text_ids [  101  8699   102  2074  2736  2019 11360  1010  2296  2487  2842  2038
  2736  2522  2015  1045  4900 

In [ ]:
print(len(input_ids))
print(len(selected_text))
print(len(start_pos))
print(len(end_pos))

input_ids = torch.Tensor(input_ids)
attention_masks = torch.Tensor(attention_masks)
token_type_ids = torch.Tensor(token_type_ids)
start_pos = torch.Tensor(start_pos)
end_pos = torch.Tensor(end_pos)

26544
26544
26544
26544


In [ ]:
print(input_ids.size(0))
print(attention_masks.size(0))
print(token_type_ids.size(0))
print(start_pos.size(0))
print(end_pos.size(0))

from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, token_type_ids, start_pos, end_pos)

train_split = int(0.9 * len(dataset))
val_split = len(dataset) - train_split

print("train_split", train_split)
print("val_split", val_split)

trainset, valset = random_split(dataset, [train_split, val_split])

26544
26544
26544
26544
26544
train_split 23889
val_split 2655


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 64

trainloader = DataLoader(trainset,
                         batch_size,
                         sampler=RandomSampler(trainset))

valloader = DataLoader(valset,
                      batch_size,
                      sampler=SequentialSampler(valset))

In [ ]:
from transformers import BertForQuestionAnswering, BertConfig, AdamW

model = BertForQuestionAnswering.from_pretrained(MODEL_NAME, output_attentions=False, output_hidden_states=False)

model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

In [ ]:
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 2

num_training_steps = epochs * len(trainloader)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
def train():
    model.train()
    training_loss = []
    eval_loss = []
    for i in range(epochs):
        batch_loss = 0
        
        for step, data in enumerate(trainloader):
            
            if step != 0 and step % 50 == 0:
                print("batch {} of {}".format(step, len(trainloader)))
            
            input_id = data[0].to(device)
            attention_mask = data[1].to(device)
            token_type_id = data[2].to(device)
            start_pos = data[3].to(device)
            end_pos = data[4].to(device)
            
            model.zero_grad()
             
            loss, start_scores, end_scores = model(input_id.long(), attention_mask.long(), token_type_id.long(), start_positions=start_pos.long(), end_positions=end_pos.long())
            
            loss.backward()
            batch_loss += loss.item()
            
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            
            optimizer.step()
            
            scheduler.step()
        average_batch_loss = batch_loss / len(trainloader)
        training_loss.append(average_batch_loss)
        print("training loss after epoch {} is {}".format(i, average_batch_loss))
        
        model.eval()

        eval_loss = []

        batch_eval_loss = 0

        for step, data in enumerate(valloader):

            if step != 0 and step % 50 == 0:
                print("batch {} of {}".format(step, len(valloader)))

            input_id = data[0].to(device)
            attention_mask = data[1].to(device)
            token_type_id = data[2].to(device)
            start_pos = data[3].to(device)
            end_pos = data[4].to(device)

            with torch.no_grad():
                loss, start_scores, end_scores = model(input_id.long(), attention_mask.long(), token_type_id.long(), start_positions=start_pos.long(), end_positions=end_pos.long())

            batch_eval_loss += loss.item()

        average_batch_eval_loss = batch_eval_loss / len(valloader)
        eval_loss.append(average_batch_eval_loss)
        print("validation loss after epoch {} is {}".format(i, average_batch_eval_loss))

train()

/pytorch/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


batch 50 of 374
batch 100 of 374
batch 150 of 374
batch 200 of 374
batch 250 of 374
batch 300 of 374
batch 350 of 374
training loss after epoch 0 is 1.1916331938881288
validation loss after epoch 0 is 0.8664918868314653
batch 50 of 374
batch 100 of 374
batch 150 of 374
batch 200 of 374
batch 250 of 374
batch 300 of 374
batch 350 of 374
training loss after epoch 1 is 0.7771890192108358
validation loss after epoch 1 is 0.8291046960013253


In [ ]:
df_test = pd.read_csv(DIR+"tweet-sentiment-extraction/test.csv")#("../input/tweet-sentiment-extraction/test.csv")

print(df_test['text'].isnull().sum())
print(df_test['sentiment'].isnull().sum())

df_test.dropna(axis = 0, how ='any',inplace=True) ;

text = df_test['text'].apply(str)
sentiment = df_test['sentiment'].apply(str)

text = text.values
print(text[0])
sentiment = sentiment.values
print(sentiment[0])


0
0
Last session of the day  http://twitpic.com/67ezh
neutral


In [ ]:
import torch

input_ids = []
attention_masks = []
token_type_ids = []

for i in range(len(text)):
    encoded = tokenizer.encode_plus(
      sentiment[i],
      text[i],
      add_special_tokens=True,
      max_length=150,
      pad_to_max_length=True,
      return_token_type_ids=True,
      return_attention_mask=True,
      return_tensors='pt'
    )
    
    input_ids.append(encoded['input_ids'])
    attention_masks.append(encoded['attention_mask'])
    token_type_ids.append(encoded['token_type_ids'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
token_type_ids = torch.cat(token_type_ids, dim=0)

print(input_ids[0])
print(attention_masks[0])
print(token_type_ids[0])

print(len(input_ids[0]))


tensor([  101,  8699,   102,  2197,  5219,  1997,  1996,  2154,  8299,  1024,
         1013,  1013,  1056,  9148, 25856,  2594,  1012,  4012,  1013,  6163,
         9351,  2232,   102,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

In [ ]:
testset = TensorDataset(input_ids, attention_masks, token_type_ids)
batch_size = 64

testloader = DataLoader(testset,
                      batch_size,
                      sampler=SequentialSampler(testset))

In [ ]:
checkpoint = {'model': model,
              'state_dict': model.state_dict(),
              'optimizer' : optimizer.state_dict()}

torch.save(checkpoint, 'checkpoint.pth')

In [ ]:
!mkdir "/content/drive/My Drive/kaggle/twitter_sentiment_extraction/QA_model_3_uncased"

In [ ]:
model.save_pretrained(DIR+"QA_model_3_uncased")

In [ ]:
df_submit = pd.read_csv(DIR+"tweet-sentiment-extraction/sample_submission.csv")#("../input/tweet-sentiment-extraction/sample_submission.csv")
def test():
    key = 0
    model.eval()

    for step, data in enumerate(testloader):

        input_id = data[0].to(device)
        attention_mask = data[1].to(device)
        token_type_id = data[2].to(device)

        with torch.no_grad():
            start_scores, end_scores = model(input_id, attention_mask, token_type_id)
        
        series_submit = pd.Series()    
        for i in range(input_id.shape[0]):
            all_tokens = tokenizer.convert_ids_to_tokens(input_id[i])
            answer = ' '.join(all_tokens[torch.argmax(start_scores[i]) : torch.argmax(end_scores[i])+1])
            print(answer)
        
#             answer = answer.detach()
#             series_submit = pd.Series(answer)
            df_submit['selected_text'][key] = answer
            key += 1

test()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  from ipykernel import kernelapp as app


last session of the day
exciting ( precisely - - skyscraper ##s gal ##ore ) . good
such a shame !
happy b ##day !
i like it ! !
that ` s great ! !
hates
blocked
and within a short time of the last clue all of them
what did you get ? my day is alright . . haven ` t done anything yet . leaving soon to my steps ##ister though !
ar ##gh total bum ##mer
i checked . we didn ` t win
. . and you ` re on twitter ! did the tavern bore you that much ?
sad ,
i feel like my phones hole is not a virgin .
i hate
miss you
cr ##amps . . .
nice songs .
i ` m going into a spiritual st ##agne ##ntation , its exploding my ego ! . i now realise , i ` m not all that great . and i ` m ok with that .
stupid
my dead
bad .
about to go to sleep
you are lame
that ##s so cool
hey peoples , don ##t you just hate being grounded ha ##ha , im just sat eating an apple and watching death note ( some anime )
huh , another scare ##point coding sunday
look who i found just for you
sweating it out on a hot humid day
guess th

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


at starbucks with my love . e ##ff school . i have work later too .
i just cried whilst watching holly ##oa ##ks . . i need a life ! lo ##l
looking forward to an exciting
term ##inator salvation . . . by myself .
dumb ##face . . . him , not u . what u up to on the w ##k ##nd ? i wanna see ##ee ##ee ##e ya !
why do i have to enter my registration details every time i update ?
i love you
ur so cute .
t ##gi ##f is right . . . i think i broke my toe last night - on my bad foot .
oh ##er ##r it ` s soon then , do you have plans ? i had a party on saturday x
very young looking dude
ignore my t ##wee ##t
cheated
breaks my ac ##hy break ##y heart
wish
im a lady and ladies don ` t move couch ##es . they direct where the couch ##es go
thanks for supporting nk ##ot ##b !
26th february
guess what ? mom adopted a kitty today ( 11 months ) his name is cork ##y
happy
thank you !
lo ##l man i got 2 1 / 2 hr ##s an ion ##t how i would ##a made it wit out my ram ##en noodles and t . v . time
not feelin

In [ ]:
df_submit

,textID,selected_text
0,f87dea47db,last session of the day
1,96d74cb729,exciting ( precisely - - skyscraper ##s gal ##...
2,eee518ae67,such a shame !
3,01082688c6,happy b ##day !
4,33987a8ee5,i like it ! !
...,...,...
3529,e5f0e6ef4b,tired
3530,416863ce47,thanks
3531,6332da480c,depression . .
3532,df1baec676,i love your videos !


In [ ]:
df.query("textID == '088c60f138'")

,textID,text,selected_text,sentiment
2,088c60f138,my boss is bullying me...,bullying me,negative


In [ ]:
sub = pd.merge(df_submit, df_test, on="textID")
sub

,textID,selected_text,text,sentiment
0,f87dea47db,last session of the day,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,exciting ( precisely - - skyscraper ##s gal ##...,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,such a shame !,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy b ##day !,happy bday!,positive
4,33987a8ee5,i like it ! !,http://twitpic.com/4w75p - I like it!!,positive
...,...,...,...,...
3529,e5f0e6ef4b,tired,"its at 3 am, im very tired but i can`t sleep ...",negative
3530,416863ce47,thanks,All alone in this old house again. Thanks for...,positive
3531,6332da480c,depression . .,I know what you mean. My little dog is sinkin...,negative
3532,df1baec676,i love your videos !,_sutra what is your next youtube video gonna b...,positive


In [ ]:
import re
print([m.start() for m in re.finditer('test', 'test test')])

def find_all(a_str, sub, start = 0):
  while start <= len(a_str):
    start = a_str.lower().find(sub, start)
    #print(start)
    if start == -1: return
    yield start
    start += 1 #len(sub) # use start += 1 to find overlapping matches

[0, 5]


In [ ]:
'''def find_all_right(a_str, sub, start = 0):
  start = len()
  while start <= len(a_str):
    start = a_str.find(sub, start)
    #print(start)
    if start == -1: return
    yield start
    start += 1 #len(sub) # use start += 1 to find overlapping matches'''

'def find_all_right(a_str, sub, start = 0):\n  start = len()\n  while start <= len(a_str):\n    start = a_str.find(sub, start)\n    #print(start)\n    if start == -1: return\n    yield start\n    start += 1 #len(sub) # use start += 1 to find overlapping matches'

In [ ]:
list(find_all('spam spam spam spam', 'spam'))

[0, 5, 10, 15]

In [ ]:
xx = "an sentence is here"
i1 = xx.find("an") + len("an") +1
i2 = xx.find("sentence")
print(i1,i2)

3 3


In [ ]:
selected_text_list = []
counter = 0
for index, row in sub.iterrows():
  txt_lst = row["selected_text"].split(" ")
  txt_lst_short = []
  for i, token in enumerate(txt_lst):
    if token[:2] == "##":
      txt_lst[i] = token[2:]

  
  start_list = list(find_all(row["text"], txt_lst[0]))
  start = start_list[0]
  
  if len(start_list) > 1 and len(txt_lst) > 1:
    second_list = list(find_all(row["text"], txt_lst[1], start_list[0]+len(txt_lst[0])))
    #print(second_list)
    for start_item in start_list:
      #print(start_item)
      if second_list[0] == start_item + len(txt_lst[0]) or second_list[0] == start_item + len(txt_lst[0]) + 1:
        #print("selected!")
        start = start_item
        break


  last_word = txt_lst[-1]
  #end = row["text"].lower().rfind(last_word) + len(last_word)
  #end = row["text"].rfind(last_word) + len(last_word)
  
  end_list = list(find_all(row["text"], txt_lst[-1]))
  end = end_list[-1] + len(last_word)
  if len(end_list) > 1 and len(txt_lst) > 1:
    secondlast_list = list(find_all(row["text"], txt_lst[-2], start)) #start+len(txt_lst[0])))
    for end_item in end_list:
      if len(secondlast_list) > 0 and (end_item + len(last_word) > start):
        if secondlast_list[-1] + len(txt_lst[-2]) == end_item or secondlast_list[-1] + len(txt_lst[-2]) + 1 == end_item:
          end = end_item + len(last_word)
          break


  final_text = row["text"][start:end]

  #while len(final_text.lower().replace(" ", "")) > len(''.join(txt_lst)):
  #  end = end_list[end_list.index(end) - 1]
    

  #if final_text.lower().replace(" ", "") != ''.join(txt_lst):
  #  final_text = " ".join(txt_lst)


  if len(final_text.lower().replace(" ", "")) != len(''.join(txt_lst)):
    counter +=1
    print("start_list: ", start_list)
    print("end_list: ", end_list)
    print(row["text"])
    print(txt_lst)
    print(start)
    print(end)
    print(final_text)
    print("-----")

  selected_text_list.append(final_text)

print(counter)

NameError: ignored

In [ ]:
"""selected_text_list = []
counter = 0
for index, row in sub.iterrows():
  txt_lst = row["selected_text"].split(" ")
  for i, token in enumerate(txt_lst):
    if token[:2] == "##":
      txt_lst[i] = token[2:]
   
  #start = row["text"].lower().find(txt_lst[0])
  start = 0 #row["text"].find(txt_lst[0])
  

  '''start_list = [m.start() for m in re.finditer(txt_lst[0], row["text"])]
  if len(start_list) > 1:
    start = start_list[1]
  else:
    start = start_list[0]'''
  start_list = list(find_all(row["text"], txt_lst[0]))
  print(start_list)
  if len(start_list) > 1 and len(txt_lst) > 1:
    second_list = list(find_all(row["text"], txt_lst[1], start_list[0]+len(txt_lst[0])))
    #print(second_list)
    for start_item in start_list:
      #print(start_item)
      if second_list[0] == start_item + len(txt_lst[0]) or second_list[0] == start_item + len(txt_lst[0]) + 1:
        #print("selected!")
        start = start_item
        break

  last_word = txt_lst[-1]
  #end = row["text"].lower().rfind(last_word) + len(last_word)
  end = row["text"].rfind(last_word) + len(last_word)
  
  end_list = list(find_all(row["text"], txt_lst[-1]))
  if len(end_list) > 1 and len(txt_lst) > 1:
    secondlast_list = list(find_all(row["text"], txt_lst[-2], start+len(txt_lst[0])))
    for end_item in end_list:
      if len(secondlast_list) > 0:
        if secondlast_list[-1] + len(txt_lst[-2]) == end_item or secondlast_list[-1] + len(txt_lst[-2]) + 1 == end_item:
          end = end_item + len(last_word)
          break


  final_text = row["text"][start:end]
  '''print("XXX ", final_text)'''
  
  '''if len(final_text.replace(" ", "")) > len(''.join(txt_lst)):
    #counter +=1
    start2 = final_text.find(txt_lst[0], start+1)
    if start2 != -1:
      start = start2
    end2 = final_text.rfind(last_word, start-1 + len(last_word)) + len(last_word)
    if end2 != -1:
      end2 = end
    final_text = final_text[start:end]
    print("XXX ", final_text)'''
  if final_text.replace(" ", "") != ''.join(txt_lst):
    counter +=1
    print(row["text"])
    print(txt_lst)
    print(start)
    print(end)
    print(final_text)
    print("-----")

  selected_text_list.append(final_text)

print(counter)"""

In [ ]:
df_submit.to_csv("submission.csv")